In [64]:
import pandas as pd
import cv2
import os
import base64
import re
# List of YouTube video URLs
video_urls = [
    "https://www.youtube.com/watch?v=JDwduqfuHjc"
]

# Create an empty list to store the data for each video
video_data = []

dfx= pd.DataFrame(video_urls,columns=["video_link"])

# function to convert image to byte string
def image_to_byte_string(image):
    _, buffer = cv2.imencode('.jpg', image)
    byte_string = base64.b64encode(buffer).decode()
    return byte_string

# function to convert video to image
def vid2frames(video_chunk_num):
    # path to input video
    part_fpath = f'{os.path.join(os.getcwd(),"temp_dir_parts")}'
    part_frame_folder_path = re.search(r"(p\d+)", video_chunk_num).group()
    part_frame_path = os.path.join(part_fpath,part_frame_folder_path)
    main_video_path =f'{os.path.join(part_fpath, video_chunk_num)}'
    
    #chunk_parts_video_fir_path p1,p2,p3,p4,p5
    print(part_frame_path)
    os.makedirs(part_frame_path, exist_ok = True)
   


    video_path = main_video_path
    #main_video_path VIDEid.mp4
    print(video_path)


    # capture video
    cap = cv2.VideoCapture(video_path)
    print(cap)
    # loop through frames and save as images
    frame_count = 1
    while cap.isOpened():
        ret, frame = cap.read()
        print("h1")
        if ret:
            frame_name= f'{part_frame_path}/{video_chunk_num}-frame{frame_count}.jpg'
            # save frame as image
            cv2.imwrite(frame_name, frame)
            print(f'{video_chunk_num}/frame{frame_count}.jpg')
            frame_count += 1
        else:
            break

    # release video capture and destroy window
    cap.release()
    cv2.destroyAllWindows()

def chunker(video_urls_list):
    # Loop through each video URL
    for url in video_urls_list:
        # Get the video ID from the URL
        video_id = url.split("=")[-1]
        print(f"NAT {video_id},,{url}")
        # Download the video using youtube-dl
        #os.system(f"youtube-dl -f 'bestvideo[ext=mp4]' -o 'video.mp4' {url}")
        # Open the video using OpenCV2
        temp_path = os.path.join(os.getcwd(),"videos")
        master_path = os.path.join(temp_path,video_id+".mp4")
        cap = cv2.VideoCapture(master_path)
        # Get the total number of frames
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        # Get the frame rate
        frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
        # Split the video into 10-second chunks with two frames per second
        chunk_size = 10 * frame_rate
        for i in range(0, total_frames, chunk_size):
            # Set the start and end frames for the chunk
            start_frame = i
            end_frame = min(i + chunk_size, total_frames)
            # Get the chunk of frames
            frames = []
            for j in range(start_frame, end_frame):
                # Read the frame
                ret, frame = cap.read()
                # Keep only the first and middle frame
                if j == start_frame or j == (start_frame + end_frame) // 2:
                    frames.append(frame)
            # Save the chunk as an mp4 file
            chunk_filename = f"{video_id}-p{int(i/chunk_size)+1}.mp4"

                # path to input video
            part_fpath = f'{os.path.join(os.getcwd(),"temp_dir_parts")}'
            os.makedirs(part_fpath, exist_ok = True)
            main_video_path =f'{os.path.join(part_fpath, chunk_filename)}'

            frame_rate_chunk = 2
            out = cv2.VideoWriter(f"{main_video_path}", cv2.VideoWriter_fourcc(*'mp4v'), frame_rate_chunk, (frames[0].shape[1], frames[0].shape[0]))
            for frame in frames:
                out.write(frame)
            out.release()
            # Add the video data to the list
            video_data.append({
                "video_id": video_id,
                "video_url": url,
                "video_parts": chunk_filename,
                "frame_rate": frame_rate_chunk,
            })
        # Release the OpenCV2 capture object
        cap.release()



def img2next(df_links):
    df=df_links
    # add columns for each frame
    print("IN img2next")
    
    part_fpath = f'{os.path.join(os.getcwd(),"temp_dir_parts")}'
    part_frame_path = os.path.join(part_fpath,part_frame_folder_path)
    
    #chunk_parts_video_fir_path p1,p2,p3,p4,p5
    print(part_frame_path)
    
    h2 = os.listdir(part_frame_path)
    h2_length = len(h2)

    video_path = main_video_path
    #main_video_path VIDEid.mp4
    print(video_path)

    print("^fiel path set^")



    for i in range(h2_length):
        df[f'frame_{i+1}'] = ''

    print(f"made columns{h2_length}")
    # iterate over each row and extract frames
    for index, row in df.iterrows():
        video_parts = row['video_parts']
        frames = []
        stopper =0
        print(index)
        for i in range(50):
            df[f'frame_{i+1}'] = ''
            video_chunk_num = video_parts
            part_frame_folder_path = re.search(r"(p\d+)", video_chunk_num).group()
            part_frame_path = os.path.join(part_fpath,part_frame_folder_path)
            main_video_path =f'{os.path.join(part_fpath, video_chunk_num)}'


            frame_path= f'{part_frame_path}\{video_chunk_num}-frame{i+1}.jpg'
            #frame_path = f'{video_parts[:-4]}-frame{i+1}.jpg'
            #frame_path = f'/home/siddharthsk7t/DL/DL-temp/frames/frame{i+1}.jpg'
            print(f"frame path {frame_path}")
            frame = cv2.imread(frame_path)
            byte_string = image_to_byte_string(frame)
            frames.append(byte_string)
            #print(f"byte string: {byte_string}")
            #print(f"fraem? string: {frames}")
            #os.remove(frame_path)
            stopper+=1
        df.loc[index, df.columns[4:]] = frames
    return df










chunker(video_urls)
# Create the final data frame
df = pd.DataFrame(video_data)
for idx in df["video_parts"]:
    print(idx)
    vid2frames(idx) 

img2next(df)







print(df)


NAT JDwduqfuHjc,,https://www.youtube.com/watch?v=JDwduqfuHjc
JDwduqfuHjc-p1.mp4
c:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts\p1
c:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts\JDwduqfuHjc-p1.mp4
< cv2.VideoCapture 000001936FC8DC90>
h1
JDwduqfuHjc-p1.mp4/frame1.jpg
h1
JDwduqfuHjc-p1.mp4/frame2.jpg
h1
JDwduqfuHjc-p2.mp4
c:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts\p2
c:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts\JDwduqfuHjc-p2.mp4
< cv2.VideoCapture 000001936FC8D5B0>
h1
JDwduqfuHjc-p2.mp4/frame1.jpg
h1
JDwduqfuHjc-p2.mp4/frame2.jpg
h1
JDwduqfuHjc-p3.mp4
c:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts\p3
c:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts\JDwduqfuHjc-p3.mp4
< cv2.VideoCapture 000001936FC8DD90>
h1
JDwduqfuHjc-p3.mp4/frame1.jpg
h1
JDwduqfuHjc-p3.mp4/frame2.jpg
h1
JDwduqfuHjc-p4.mp4
c:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts\p4
c:\Users\SkyLake\Documents\Python 2.0\Proj02b\tem

UnboundLocalError: local variable 'part_frame_folder_path' referenced before assignment

In [65]:
df

,video_id,video_url,video_parts,frame_rate
0,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p1.mp4,2
1,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p2.mp4,2
2,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p3.mp4,2
3,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p4.mp4,2
4,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p5.mp4,2
5,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p6.mp4,2
6,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p7.mp4,2


In [55]:
img2next()

IN img2next
c:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts\p5
c:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts\JDwduqfuHjc-p5.mp4
^fiel path set^
made columns2
0
frame path c:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts\p1\JDwduqfuHjc-p1.mp4-frame1.jpg
frame path c:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts\p1\JDwduqfuHjc-p1.mp4-frame2.jpg
1
frame path c:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts\p2\JDwduqfuHjc-p2.mp4-frame1.jpg
frame path c:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts\p2\JDwduqfuHjc-p2.mp4-frame2.jpg
2
frame path c:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts\p3\JDwduqfuHjc-p3.mp4-frame1.jpg
frame path c:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts\p3\JDwduqfuHjc-p3.mp4-frame2.jpg
3
frame path c:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts\p4\JDwduqfuHjc-p4.mp4-frame1.jpg
frame path c:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts

In [51]:
vid2frames("JDwduqfuHjc-p7.mp4")

c:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts\p7
c:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts\JDwduqfuHjc-p7.mp4
< cv2.VideoCapture 000001936FC8DB90>
h1
JDwduqfuHjc-p7.mp4/frame1.jpg
h1
JDwduqfuHjc-p7.mp4/frame2.jpg
h1


In [56]:
df

,video_id,video_url,video_parts,frame_rate,frame_1,frame_2
0,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p1.mp4,2,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
1,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p2.mp4,2,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
2,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p3.mp4,2,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
3,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p4.mp4,2,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
4,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p5.mp4,2,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
5,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p6.mp4,2,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
6,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p7.mp4,2,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
